In [1]:
from datetime import datetime as dt
import pandas as pd
import time
import warnings

pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

In [ ]:
url_by_city = 'https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/city_market_tracker.tsv000.gz'

def extract_data(url):
    df = pd.read_csv(url, compression='gzip', sep= '\t' )
    return df

df = extract_data(url_by_city)
df.head()

In [2]:
print('yes')

yes


In [1]:
import pandas as pd

url_by_city = 'https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/city_market_tracker.tsv000.gz'

def extract_data(url, chunk_number=0, chunk_size=10000):
    # Create a chunk iterator
    chunks = pd.read_csv(url, compression='gzip', sep='\t', chunksize=chunk_size)

    for i, chunk in enumerate(chunks):
        if i == chunk_number:
            return chunk
    return pd.DataFrame()


df = extract_data(url_by_city, chunk_number=0, chunk_size=1000)
print(df.head())


  PERIOD_BEGIN  PERIOD_END  PERIOD_DURATION REGION_TYPE  REGION_TYPE_ID  \
0   2015-10-01  2015-10-31               30       place               6   
1   2014-04-01  2014-04-30               30       place               6   
2   2017-10-01  2017-10-31               30       place               6   
3   2024-07-01  2024-07-31               30       place               6   
4   2022-01-01  2022-01-31               30       place               6   

   TABLE_ID  IS_SEASONALLY_ADJUSTED            REGION          CITY  \
0      8591                   False  Indian Trail, NC  Indian Trail   
1      8225                   False   Hainesville, IL   Hainesville   
2     10716                   False     Maysville, NC     Maysville   
3      3140                   False      Chewelah, WA      Chewelah   
4     11322                   False       Lombard, IL       Lombard   

            STATE  ... SOLD_ABOVE_LIST_YOY PRICE_DROPS  PRICE_DROPS_MOM  \
0  North Carolina  ...            0.038406    0

I used chuksize to limit the amount of data that needs to be processed on my notebook instance

In [2]:
print(df.columns.tolist())

['PERIOD_BEGIN', 'PERIOD_END', 'PERIOD_DURATION', 'REGION_TYPE', 'REGION_TYPE_ID', 'TABLE_ID', 'IS_SEASONALLY_ADJUSTED', 'REGION', 'CITY', 'STATE', 'STATE_CODE', 'PROPERTY_TYPE', 'PROPERTY_TYPE_ID', 'MEDIAN_SALE_PRICE', 'MEDIAN_SALE_PRICE_MOM', 'MEDIAN_SALE_PRICE_YOY', 'MEDIAN_LIST_PRICE', 'MEDIAN_LIST_PRICE_MOM', 'MEDIAN_LIST_PRICE_YOY', 'MEDIAN_PPSF', 'MEDIAN_PPSF_MOM', 'MEDIAN_PPSF_YOY', 'MEDIAN_LIST_PPSF', 'MEDIAN_LIST_PPSF_MOM', 'MEDIAN_LIST_PPSF_YOY', 'HOMES_SOLD', 'HOMES_SOLD_MOM', 'HOMES_SOLD_YOY', 'PENDING_SALES', 'PENDING_SALES_MOM', 'PENDING_SALES_YOY', 'NEW_LISTINGS', 'NEW_LISTINGS_MOM', 'NEW_LISTINGS_YOY', 'INVENTORY', 'INVENTORY_MOM', 'INVENTORY_YOY', 'MONTHS_OF_SUPPLY', 'MONTHS_OF_SUPPLY_MOM', 'MONTHS_OF_SUPPLY_YOY', 'MEDIAN_DOM', 'MEDIAN_DOM_MOM', 'MEDIAN_DOM_YOY', 'AVG_SALE_TO_LIST', 'AVG_SALE_TO_LIST_MOM', 'AVG_SALE_TO_LIST_YOY', 'SOLD_ABOVE_LIST', 'SOLD_ABOVE_LIST_MOM', 'SOLD_ABOVE_LIST_YOY', 'PRICE_DROPS', 'PRICE_DROPS_MOM', 'PRICE_DROPS_YOY', 'OFF_MARKET_IN_TWO_WEE

Now, I want to clean up the dataframe and build the function that cleans up the dataframe. 

In [3]:
df.head()

,PERIOD_BEGIN,PERIOD_END,PERIOD_DURATION,REGION_TYPE,REGION_TYPE_ID,TABLE_ID,IS_SEASONALLY_ADJUSTED,REGION,CITY,STATE,...,SOLD_ABOVE_LIST_YOY,PRICE_DROPS,PRICE_DROPS_MOM,PRICE_DROPS_YOY,OFF_MARKET_IN_TWO_WEEKS,OFF_MARKET_IN_TWO_WEEKS_MOM,OFF_MARKET_IN_TWO_WEEKS_YOY,PARENT_METRO_REGION,PARENT_METRO_REGION_METRO_CODE,LAST_UPDATED
0,2015-10-01,2015-10-31,30,place,6,8591,False,"Indian Trail, NC",Indian Trail,North Carolina,...,0.038406,0.224066,-0.011228,0.023309,0.291667,-0.084804,0.034524,"Charlotte, NC",16740,2025-04-18 14:40:47.550 Z
1,2014-04-01,2014-04-30,30,place,6,8225,False,"Hainesville, IL",Hainesville,Illinois,...,0.500000,0.333333,0.166667,NaN,NaN,NaN,NaN,"Lake County, IL",29404,2025-04-18 14:40:47.550 Z
2,2017-10-01,2017-10-31,30,place,6,10716,False,"Maysville, NC",Maysville,North Carolina,...,0.500000,NaN,NaN,NaN,0.000000,0.000000,0.000000,"New Bern, NC",35100,2025-04-18 14:40:47.550 Z
3,2024-07-01,2024-07-31,30,place,6,3140,False,"Chewelah, WA",Chewelah,Washington,...,0.000000,0.250000,-0.057692,-0.659091,0.500000,-0.166667,0.277778,"Spokane, WA",44060,2025-04-18 14:40:47.550 Z
4,2022-01-01,2022-01-31,30,place,6,11322,False,"Lombard, IL",Lombard,Illinois,...,-0.018092,0.134615,-0.028175,-0.280019,0.241379,-0.175287,-0.136398,"Chicago, IL",16984,2025-04-18 14:40:47.550 Z


In [11]:
rows = len(df)
columns = len(df.columns)

print(f'Total number of rows: {rows}')
print(f'Total number of columns: {columns}')

Total number of rows: 1000
Total number of columns: 58


In [7]:
columns_to_keep = [
    'PERIOD_BEGIN', 'PERIOD_END', 'CITY', 'STATE', 'REGION', 'PROPERTY_TYPE', 'IS_SEASONALLY_ADJUSTED',
    'MEDIAN_SALE_PRICE', 'MEDIAN_SALE_PRICE_MOM', 'MEDIAN_SALE_PRICE_YOY',
    'MEDIAN_LIST_PRICE',
    'MEDIAN_PPSF',
    'HOMES_SOLD', 'HOMES_SOLD_MOM', 'HOMES_SOLD_YOY',
    'NEW_LISTINGS',
    'INVENTORY', 'MONTHS_OF_SUPPLY',
    'MEDIAN_DOM',
    'AVG_SALE_TO_LIST',
    'SOLD_ABOVE_LIST'
]

In [8]:
df[columns_to_keep].head()

,PERIOD_BEGIN,PERIOD_END,CITY,STATE,REGION,PROPERTY_TYPE,IS_SEASONALLY_ADJUSTED,MEDIAN_SALE_PRICE,MEDIAN_SALE_PRICE_MOM,MEDIAN_SALE_PRICE_YOY,...,MEDIAN_PPSF,HOMES_SOLD,HOMES_SOLD_MOM,HOMES_SOLD_YOY,NEW_LISTINGS,INVENTORY,MONTHS_OF_SUPPLY,MEDIAN_DOM,AVG_SALE_TO_LIST,SOLD_ABOVE_LIST
0,2015-10-01,2015-10-31,Indian Trail,North Carolina,"Indian Trail, NC",Single Family Residential,False,223000.0,0.152455,0.140665,...,100.671141,69,-0.028169,0.15000,75.0,241.0,3.5,85.0,0.984053,0.188406
1,2014-04-01,2014-04-30,Hainesville,Illinois,"Hainesville, IL",Condo/Co-op,False,82625.0,-0.220512,0.832040,...,67.226870,2,1.000000,1.00000,NaN,6.0,3.0,83.0,1.194046,0.500000
2,2017-10-01,2017-10-31,Maysville,North Carolina,"Maysville, NC",All Residential,False,67256.0,-0.115053,-0.148658,...,41.009756,1,0.000000,-0.50000,3.0,12.0,12.0,41.0,1.033118,1.000000
3,2024-07-01,2024-07-31,Chewelah,Washington,"Chewelah, WA",Single Family Residential,False,317500.0,0.058333,-0.290503,...,237.238371,4,-0.200000,0.00000,5.0,12.0,3.0,25.0,0.991608,0.000000
4,2022-01-01,2022-01-31,Lombard,Illinois,"Lombard, IL",Single Family Residential,False,340000.0,0.014925,0.101215,...,213.068182,19,-0.486486,-0.40625,23.0,52.0,2.7,45.0,0.977684,0.263158


In [13]:
df_cleaned = df[columns_to_keep]

In [14]:
df_cleaned.dropna()
rows2 = len(df_cleaned)

print(f'Total number of rows: {rows2}')

Total number of rows: 1000


So there is no missing data in the sample listed here. But we'll still have to add this cleanup in the function itself in order to avoid complications. 

In [15]:
# lets check the datatypes of the columns I intend to keep for the analysis

df_cleaned.dtypes.to_frame(name='Data Type')

,Data Type
PERIOD_BEGIN,object
PERIOD_END,object
CITY,object
STATE,object
REGION,object
PROPERTY_TYPE,object
IS_SEASONALLY_ADJUSTED,bool
MEDIAN_SALE_PRICE,float64
MEDIAN_SALE_PRICE_MOM,float64
MEDIAN_SALE_PRICE_YOY,float64


So I need to change the datatype of just 'PERIOD_BEGIN' and 'PERIOD_END'

In [17]:
df_cleaned['PERIOD_BEGIN'] = pd.to_datetime(df_cleaned['PERIOD_BEGIN'])
df_cleaned['PERIOD_END'] = pd.to_datetime(df_cleaned['PERIOD_END'])

df_cleaned.dtypes.to_frame(name='Data Type')

/tmp/ipykernel_7940/4278986289.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['PERIOD_BEGIN'] = pd.to_datetime(df_cleaned['PERIOD_BEGIN'])
/tmp/ipykernel_7940/4278986289.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['PERIOD_END'] = pd.to_datetime(df_cleaned['PERIOD_END'])


,Data Type
PERIOD_BEGIN,datetime64[ns]
PERIOD_END,datetime64[ns]
CITY,object
STATE,object
REGION,object
PROPERTY_TYPE,object
IS_SEASONALLY_ADJUSTED,bool
MEDIAN_SALE_PRICE,float64
MEDIAN_SALE_PRICE_MOM,float64
MEDIAN_SALE_PRICE_YOY,float64


Now, all the datatypes looks okay

### The final cleaning function

In [18]:
def data_cleanup(url):
    df = extract_data(url, chunk_number=0, chunk_size=1000) # this would be changed when used in the actual task on airflow.
    
    # remove commas from the city column
    df['CITY'] = df['CITY'].str.replace(',', '')

    #define the columns that I want in the cleanedup data (I only chose columns that are necessary for the analysis I want to perform at the end of the project)
    columns_to_keep = [
    'PERIOD_BEGIN', 'PERIOD_END', 'CITY', 'STATE', 'REGION', 'PROPERTY_TYPE', 'IS_SEASONALLY_ADJUSTED',
    'MEDIAN_SALE_PRICE', 'MEDIAN_SALE_PRICE_MOM', 'MEDIAN_SALE_PRICE_YOY',
    'MEDIAN_LIST_PRICE',
    'MEDIAN_PPSF',
    'HOMES_SOLD', 'HOMES_SOLD_MOM', 'HOMES_SOLD_YOY',
    'NEW_LISTINGS',
    'INVENTORY', 'MONTHS_OF_SUPPLY',
    'MEDIAN_DOM',
    'AVG_SALE_TO_LIST',
    'SOLD_ABOVE_LIST'
    ]
    df = df[columns_to_keep]

    # now, drop all missing values 
    df = df.dropna()

    #change the dataype of the date columns in the table 
    df['PERIOD_BEGIN'] = pd.to_datetime(df['PERIOD_BEGIN'])
    df['PERIOD_END'] = pd.to_datetime(df['PERIOD_END'])

    # Now, lets print the total number of rows and columns. This would be on the logs in airflow. 
    rows = len(df)
    columns = len(df.columns)
    
    print(f'Total number of rows: {rows}')
    print(f'Total number of columns: {columns}')
    return df

    

In [19]:
url_by_city = 'https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/city_market_tracker.tsv000.gz'

# Call the cleanup function with the URL
cleaned_df = data_cleanup(url_by_city)
cleaned_df.head()

Total number of rows: 658
Total number of columns: 21


AttributeError: 'NoneType' object has no attribute 'head'